In [7]:
import numpy as np
import ArtistExtractor as AE
import pandas as pd
import glob
import os
import LocationUtils as LU
from ExtractGenreClubs import SplitLineup
from ast import literal_eval
import moreFunction

In [8]:
def getAllArtists():
    file = "ClubData/ClubDataGeo/ClubDataFrame.csv"
    ClubDataFrame = pd.read_csv(file,sep=",")
    ClubDataFrame["Genre"] = None
    ClubDataFrame["location"] = None
    ClubDataFrame["event_style"] = 1
    ClubDataFrame = ClubDataFrame[["ClubName","club","Address","location","EventName","Date","LineUp","Genre","Location"]]
    ClubDataFrame.columns = ["place","event_style","address","location","event","date","artists","genre","coordinates"]
    ClubDataFrame

    file = "events.ch/EventsChData/completeWithCoordinates.csv"
    EventsDataFrame = pd.read_csv(file)
    EventsDataFrame["address"] = None
    EventsDataFrame["place"] = None
    EventsDataFrame["event_style"] = 0
    EventsDataFrame = EventsDataFrame[["place","event_style","address","location","event","date","artists","genre","coordinates"]]
    EventsDataFrame

    
    ##ADD HERE CODE FOR ROUTE DES FESTIVAL
    '''
    file = ##Add path
    RouteFestivalDataFrame = pd.read_csv(file)
    RouteFestivalDataFrame["address"] = None
    RouteFestivalDataFrame["place"] = None
    RouteFestivalDataFrame["event_style"] = 2
    RouteFestivalDataFrame = RouteFestivalDataFrame[["place","event_style","address","location","event","date","artists","genre","coordinates"]]
    RouteFestivalDataFrame
    '''
    
    
    Dataframe = pd.DataFrame()
    Dataframe = Dataframe.append(EventsDataFrame)
    Dataframe = Dataframe.append(ClubDataFrame)
    ## Dataframe = Dataframe.append(RouteFestivalDataFrame)
    Dataframe.to_csv("FullData/EventsAndClubs.csv", encoding="utf-8")

    ArtistsDataFrame = pd.DataFrame()
    i=0
    print(Dataframe.shape)
    for index,row in Dataframe.iterrows():
        lineup = row["artists"] #get line-up
        artists = literal_eval(lineup)
        for a in artists:
            a = a.split("(")[0]#Remove parenthesis at end of artist (record label, live act..)

            artist_list = []
            artist_list.append(a)
            cleaned_artist = moreFunction.clean_artists(artist_list)
            if(len(cleaned_artist)>0):
                a = cleaned_artist[0]
            a = pd.Series(a.strip())
            ArtistsDataFrame = ArtistsDataFrame.append(a,ignore_index=True)
        i+=1
        if(i%5000==0):
            print(i)

    ArtistsDataFrame.columns=["artist"]
    ArtistsDataFrame.to_csv("FullData/ArtistsList.csv",encoding="utf-8")
    return ArtistsDataFrame

In [9]:
def downloadGenres(artistsDF,begin=0,end=100000):

    Artists = artistsDF.copy()
    Artists = Artists.drop_duplicates().reset_index()[["artist"]]
    Artists.columns=["artist"]
    Artists["genres_spotify"]=None
    Artists["genres_ra"]=None
    Artists["main_genres"]=None
    Artists["top3_genres"]=None
    Artists["genre"]=None    
    
    if(end>Artists.shape[0]):
        end = Artists.shape[0]
    
    path = "FullData/ArtistDataframe_"+str(begin)+"_"+str(end)+".csv"
    print("Downloading genres of "+str(end-begin)+" artist...")
    last_read = 0
    for i in range(begin,end):
        S = Artists[Artists.index==i]
        artist = S["artist"].values[0]
        genres_spotify = AE.getGenresFromSpotify(artist)
        
        if(len(genres_spotify)<1):
            genres_spotify = None
        Artists.loc[Artists.index==i,"genres_spotify"] = str(genres_spotify)

        if(i%300==0):
            Artists.to_csv(path,encoding="utf-8")
            print(str(i)+": Saved under "+path)
    
    Artists.to_csv(path,encoding="utf-8")
    
    print(str(i)+": Saved under "+path)
    last_read = i

    print("Last read = "+str(last_read))
    return Artists

In [10]:
def generateDictionnary(artistsDF):
    df = artistsDF.copy()

    i=0
    AllGenresDownloaded = []
    print("Populating dictionnary with other genres:")
    print("Reading all genres from dataframe : "+path)
    for id,row in df.iterrows():
        listGenres = literal_eval(row[2])
        if(listGenres!=None):
            AllGenresDownloaded+=listGenres
    print("All genre extracted from dataframe. Updating dictionnary..")
    dic = AE.createDictionnary()
    dic = AE.updateDictionnary(AllGenresDownloaded,dic,debug=True)
    LU.saveDictionary(dic,"FullData/AllGenresDic",enc="utf-8")
    print("Dictionnary of genre saved.")
    
    return dic

In [11]:
def downloadGenresWikipedia(str_file,dictionnary,begin=0,end=100000):

    Artists = pd.read_csv(str_file,index_col=0)
    
    if(end>Artists.shape[0]):
        end = Artists.shape[0]
    
    path = "FullData/ArtistDataframe_"+str(begin)+"_"+str(end)+".csv"
    print("Downloading genres of "+str(end-begin)+" artist...")
    last_read = 0
    for i in range(begin,end):
        S = Artists[Artists.index==i]
        artist = S["artist"].values[0]
        genres_ra = AE.getGenresFromWikipedia(artist,dictionnary)
        if(len(genres_ra)<1):
            genres_ra = None
        Artists.loc[Artists.index==i,"genres_ra"] = str(genres_ra)

        if(i%300==0):
            Artists.to_csv(path,encoding="utf-8")
            print(str(i)+": Saved under "+path)
    
    Artists.to_csv(path,encoding="utf-8")
    
    print(str(i)+": Saved under "+path)
    last_read = i

    print("Last read = "+str(last_read))
    return Artists

In [5]:
ArtistsDF = getAllArtists()

(38846, 9)
5000
10000
15000
20000
25000
30000
35000


In [13]:
ArtistsDF = downloadGenres(ArtistsDF)
ArtistsDF

0: Saved under FullData/ArtistDataframe_0_44827.csv
300: Saved under FullData/ArtistDataframe_0_44827.csv
600: Saved under FullData/ArtistDataframe_0_44827.csv
900: Saved under FullData/ArtistDataframe_0_44827.csv
1200: Saved under FullData/ArtistDataframe_0_44827.csv
1500: Saved under FullData/ArtistDataframe_0_44827.csv
1800: Saved under FullData/ArtistDataframe_0_44827.csv
2100: Saved under FullData/ArtistDataframe_0_44827.csv
2400: Saved under FullData/ArtistDataframe_0_44827.csv
2700: Saved under FullData/ArtistDataframe_0_44827.csv
3000: Saved under FullData/ArtistDataframe_0_44827.csv
3300: Saved under FullData/ArtistDataframe_0_44827.csv
3600: Saved under FullData/ArtistDataframe_0_44827.csv
3900: Saved under FullData/ArtistDataframe_0_44827.csv
4200: Saved under FullData/ArtistDataframe_0_44827.csv
4500: Saved under FullData/ArtistDataframe_0_44827.csv
4800: Saved under FullData/ArtistDataframe_0_44827.csv
5100: Saved under FullData/ArtistDataframe_0_44827.csv
5400: Saved unde

,artist,genres_spotify,genres_ra,main_genres,top3_genres,genre
0,Andy Gruber Band,None,None,None,None,None
1,DJ Raykard,None,None,None,None,None
2,Martin Sander,None,None,None,None,None
3,Bernhard Hörler,None,None,None,None,None
4,Oomph,"['german metal', 'industrial metal', 'industri...",None,None,None,None
5,Plusmacher,"['deep german hip hop', 'german hip hop']",None,None,None,None
6,Konstantin Scherbakov,"['classical piano', 'concert piano']",None,None,None,None
7,The Bosshoss,"['comic', 'german pop', 'german pop rock']",None,None,None,None
8,The Sore Loosers,None,None,None,None,None
9,Viper Sniper,None,None,None,None,None
